# Amazon Personalize AWS 사용자 개인화 + 상황별 추천 예


## 소개 <a class="anchor" id="intro"></a>

대부분의 경우, Amazon Personalize(레시피라고 함)의 알고리즘은 여기서 설명하는 다양한 작업을 처리합니다.

1. **사용자 개인화** - 이전 사용자 상호 작용에 따라 항목을 추천합니다.
1. **개인별 순위** - 항목 모음을 선택한 다음 HRNN과 유사한 접근 방식을 사용하여 항목 순서를 지정합니다.
1. **SIMS(유사 항목)** - 특정 항목이 주어지면 사용자가 상호 작용하는 다른 항목도 추천합니다.

사용 사례에 관계없이, 모든 알고리즘은 3가지 핵심 속성으로 정의된 사용자-항목-상호 작용 데이터에 대한 학습 기반을 공유합니다.

1. **UserID** - 상호 작용한 사용자
1. **ItemID** - 사용자가 상호 작용한 항목
1. **타임스탬프** - 상호 작용이 발생한 시간


## 데이터 세트 또는 데이터 소스 선택 <a class="anchor" id="source"></a>
[맨 위로 이동](#top)

앞서 언급했듯이, 사용자-항목-상호 작용 데이터는 서비스를 시작하기 위한 핵심 속성입니다. 즉, 이러한 종류의 데이터를 생성하는 사용 사례를 찾아야 합니다. 몇 가지 일반적인 예는 다음과 같습니다.

1. 비디오 온디맨드 애플리케이션
1. 전자 상거래 플랫폼
1. 소셜 미디어 집계기/플랫폼

Personalize에 적합한 문제의 범위를 지정하는 데 적용되는 몇 가지 지침이 있습니다. [공식 한도](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)는 조금 더 낮지만 아래의 값을 시작점으로 사용하면 유용합니다.

* 인증된 사용자
* 50명 이상의 고유한 사용자
* 100개 이상의 고유한 항목
* 사용자당 24건 이상의 상호 작용 

대부분의 경우 이는 쉽게 달성할 수 있으며 한 범주에서 값이 기준에 못 미치면 다른 범주에서 더 많은 수를 확보하여 보충할 수 있습니다.

일반적으로 데이터는 Personalize에 사용하기에 완벽한 형식으로 제공되지 않으며, 올바르게 구성되기 위해서는 약간의 수정이 필요합니다. 이 노트북에서는 그러한 모든 과정을 안내합니다.

먼저, 항공사 리뷰 데이터 세트를 사용해 보겠습니다. Skytrax(www.airlinequality.com )에 게시된 모든 사용자 리뷰를 모아 만든 데이터 세트입니다. 이 데이터는 https://github.com/quankiquanki/skytrax-reviews-dataset 에서 확인할 수 있습니다. 

In [274]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

### 데이터 세트 가져오기 및 탐색

In [275]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


--2020-06-18 16:33:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv’

airline.csv         100%[===================>]  33.14M   113MB/s    in 0.3s    

2020-06-18 16:33:45 (113 MB/s) - ‘airline.csv’ saved [34752262/34752262]



In [276]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


여기서 보듯이 데이터 세트에는 Amazon Personalize에서 필요한 데이터 세트를 생성하는 데 사용할 수 있는 열이 많이 있습니다.

먼저 데이터 세트의 복사본을 2개 만듭니다.

In [277]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## 상호 작용 데이터 세트 만들기

상호 작용 데이터 세트를 만들어 보겠습니다. 다음 단계에 따라 만듭니다.

- 불필요한 열 제거
- 이벤트 유형을 설명하는 새 열 생성
- 열 이름을 Amazon Personalize 가져오기 작업에 맞는 보다 일반적인 명명 규칙에 따라 변경



In [278]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize는 컨텍스트(예: 디바이스 유형, 위치, 시간 등)에 맞는 추천을 생성하여 추천의 정확도를 높일 수 있는 **상황별 추천**을 지원합니다. 상황별 정보는 사용자의 과거 상호 작용을 알 수 없는 신규 사용자/신원이 확인되지 않은 사용자를 위한 개인화에도 유용합니다.

이 예에서는 **좌석 유형**을 컨텍스트로 사용하여 사용자에게 최적의 항공사를 추천합니다. 추천을 받을 때 컨텍스트로서 어떤 가치를 전달할 수 있는지 알아보겠습니다.


In [279]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

보시다시피 데이터 세트의 현재 **타임스탬프** 값은 문자열입니다. Amazon Personalize에는 UNIX 형식의 타임스탬프 값이 필요합니다. 임의의 타임스탬프 값을 가져와서 Unix 형식으로 변환해보겠습니다.

In [280]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079568000.0


이제 타임스탬프 열의 모든 값에 대해 동일한 변환을 수행합니다.

In [281]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428624e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420416e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410653e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409962e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402877e+09,Economy,4.0,RATING


데이터 세트 속성 몇 가지를 살펴보겠습니다.

In [282]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373950e+09,6.039527
std,5.771909e+07,3.214680
min,0.000000e+00,1.000000
25%,1.350864e+09,3.000000
50%,1.389658e+09,7.000000
75%,1.412122e+09,9.000000
max,1.438474e+09,10.000000


Null 값이 있나요?

In [283]:
a_interactions_df.isnull().values.any()

True

Null 값을 삭제하고 Null 값이 없는지 확인합니다.

In [284]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

이제 Amazon Personalize에 사용할 수 있는 데이터가 준비되었으므로 로컬에 파일로 저장해 보겠습니다.

In [285]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## 사용자 데이터 세트 만들기

사용자 데이터 세트를 만들어보겠습니다. 다음 단계에 따라 만듭니다.

- 불필요한 열 제거
- 사용자 메타데이터로서 국적을 설명하는 새 열 생성
- 열 이름을 Amazon Personalize 가져오기 작업에 맞는 보다 일반적인 명명 규칙에 따라 변경


In [286]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>
[맨 위로 이동](#top)

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

In [288]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 버킷 이름은 전역적으로 고유합니다. 고유한 버킷 이름을 만들기 위해 아래 코드는 AWS 계정 번호에 `personalizepoc` 문자열을 추가합니다. 그런 다음 이전 셀에서 확인된 리전에 이 이름으로 버킷을 만듭니다.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-user-personalization-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다. 

In [294]:
interactions_filename = data_dir + '/a_interactions.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [295]:
user_metadata_file = data_dir + '/a_users.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## 데이터 세트 그룹 및 상호 작용 데이터 세트 생성 <a class="anchor" id="group_dataset"></a>
[맨 위로 이동](#top)

Amazon Personalize에서 격리 및 추상화의 최상위 수준은 데이터 세트 그룹입니다. 이러한 데이터 세트 그룹 중 하나에 저장된 정보는 다른 데이터 세트 그룹이나 거기에서 생성된 모델에 영향을 주지 않으며 완전히 격리됩니다. 이를 통해 많은 실험을 실행할 수 있으며, 이는 모델을 비공개 상태로 사용자의 데이터에 대해서만 완벽하게 훈련된 상태로 유지하기 위한 방법 중 하나입니다.

이전에 준비된 데이터를 가져오기 전에, 데이터 세트 그룹이 필요하고 해당 데이터 세트 그룹에 상호 작용을 처리하는 데이터 세트를 추가해야 합니다.

데이터 세트 그룹은 다음과 같은 유형의 정보를 저장할 수 있습니다.

* 사용자-항목-상호 작용
* 이벤트 스트림(실시간 상호 작용)
* 사용자 메타데이터
* 항목 메타데이터

상호 작용 데이터의 데이터 세트 그룹과 데이터 세트를 만들기 전에, 사용자 환경이 Amazon Personalize와 정상적으로 통신할 수 있는지 검증해 보겠습니다.

In [293]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

### 데이터 세트 그룹 생성

다음 셀은 airlines-dataset-group+접두사를 이름으로 사용하여 새 데이터 세트 그룹을 만듭니다.

In [64]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/airlines-dataset-group-55035",
  "ResponseMetadata": {
    "RequestId": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:14:12 GMT",
      "x-amzn-requestid": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


데이터 세트 그룹을 사용하려면 먼저 그룹이 활성화되어 있어야 합니다. 활성화하는 데에는 1~2분이 걸립니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 데이터 세트 그룹의 상태를 확인합니다.

In [65]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


이제 데이터 세트 그룹을 만들었으므로 상호 작용 데이터의 데이터 세트를 만들 수 있습니다.

# 데이터 세트 생성

### 상호 작용 데이터 세트

먼저 업로드하려는 데이터 세트의 유형을 Amazon Personalize에 알려주는 스키마를 정의합니다. 스키마에는 데이터 세트 유형에 따라 몇 가지 예약 키워드와 필수 키워드가 필요합니다. 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)를 참조하세요.

여기서는 `USER_ID`, `ITEM_ID`, `TIMESTAMP`, `CABIN_TYPE`, `EVENT_TYPE`, `EVENT_VALUE` 및 `TIMESTAMP` 필드가 필요한 상호 작용 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.

In [55]:
schema_name="airlines-interaction-schema-"+suffix

In [56]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-interaction-schema-55035",
  "ResponseMetadata": {
    "RequestId": "4e045a61-d479-485c-93ff-6072076ccaa9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:10:34 GMT",
      "x-amzn-requestid": "4e045a61-d479-485c-93ff-6072076ccaa9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/airlines-dataset-group-55035/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "968e6cac-310a-4889-8243-e86ef90696ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:15:14 GMT",
      "x-amzn-requestid": "968e6cac-310a-4889-8243-e86ef90696ed",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 사용자 데이터 세트

여기서는 `USER_ID` 및 `NATIONALITY` 필드가 필요한 사용자 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.


In [62]:
metadata_schema_name="airlines-users-schema-"+suffix

In [63]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-users-schema-55035",
  "ResponseMetadata": {
    "RequestId": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:13:50 GMT",
      "x-amzn-requestid": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

### S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

In [69]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### IAM 역할 생성

Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. IAM 역할을 생성하고 필요한 정책을 연결해보겠습니다. 아래의 코드는 과도하게 권한을 부여하는 정책을 연결합니다. 프로덕션 애플리케이션에는 보다 제한적인 정책을 사용하세요.

In [70]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeS3Role-55035


# 데이터 세트 가져오기 작업 생성

## 상호 작용 데이터 가져오기 <a class="anchor" id="import"></a>

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [109]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "d118cf11-b568-4767-99d5-30a15871981a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:38 GMT",
      "x-amzn-requestid": "d118cf11-b568-4767-99d5-30a15871981a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## 사용자 데이터 가져오기 <a class="anchor" id="import"></a>

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [110]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-users-metadata-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "679d9401-568d-45e0-ba8c-df8f1574228d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:40 GMT",
      "x-amzn-requestid": "679d9401-568d-45e0-ba8c-df8f1574228d",
      "content-length": "136",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 데이터 세트 가져오기 작업이 활성 상태가 될 때까지 기다리기

데이터 세트를 사용하려면 먼저 가져오기 작업이 활성 상태여야 합니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 가져오기 작업의 상태를 확인합니다.

데이터 세트의 크기에 따라 데이터를 가져오는 데 시간이 걸릴 수 있습니다. 이 데모에서는 데이터 가져오기 작업을 미리 실행했습니다.

In [111]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [112]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


데이터 세트 가져오기가 활성화되면 AWS 사용자 개인화 레시피를 사용하여 모델 구축을 시작할 준비가 된 것입니다.

## 솔루션 생성 <a class="anchor" id="solutions"></a>
[맨 위로 이동](#top)

이 노트북에서는 다음 레시피로 솔루션을 만듭니다.

1. aws-user-personalization


Amazon Personalize에서는 알고리즘의 특정 변형 버전을 레시피라고 합니다. 레시피마다 적용되는 상황이 다릅니다. 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 특정 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열하겠습니다. 이렇게 하면 모델을 선택하고 모델을 구축하는 데 사용할 수 있습니다.

In [113]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


출력은 도입부에 언급된 모든 알고리즘의 JSON 표현일 뿐입니다.

다음으로 구체적인 레시피를 선택하여 그 레시피로 모델을 만듭니다.

### AWS 사용자 개인화

AWS 사용자 개인화는 여러분이 사용할 수 있는 고급 추천 모델 중 하나이며 사용자 행동에 따라 추천을 실시간으로 업데이트할 수 있습니다. 또한 협업 필터링과 같은 다른 접근 방식을 능가하는 경우가 많습니다. 이 레시피는 훈련 시간이 가장 오래 걸리는데, 우선 이 레시피부터 시작하겠습니다.

이 사용 사례의 경우 항공사 리뷰 데이터를 사용합니다. AWS 사용자 개인화를 사용하여 사용자의 이전 아티스트 태깅 행동에 따라 항공사를 사용자에게 추천할 수 있습니다. 태깅 데이터를 사용하여 사용자와 아티스트 간의 긍정적인 상호 작용을 표현했다는 것을 기억하실 겁니다.

먼저 위의 레시피 목록에서 ARN을 찾아 레시피를 선택합니다.

In [114]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### 솔루션 생성

먼저 레시피를 사용하여 솔루션을 만듭니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

HPO가 활성화되어 있는 것을 확인할 수 있습니다. 이는 다음의 경우에 유용합니다.


In [ ]:
create_solution_response = personalize.create_solution(
    name = "airlines-user-personalization-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 40분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [117]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "ResponseMetadata": {
    "RequestId": "148a0fbd-5465-4619-ac90-449c0ef23b73",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:32 GMT",
      "x-amzn-requestid": "148a0fbd-5465-4619-ac90-449c0ef23b73",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## 솔루션 버전 평가 <a class="anchor" id="eval"></a>
[맨 위로 이동](#top)

한 시간이면 이 노트북의 모든 솔루션을 훈련할 수 있습니다. 훈련이 진행되는 동안 시간을 할애해 다양한 알고리즘(레시피)과 그 동작에 대해 자세히 읽어보시기 바랍니다. 또한 이 시간은 데이터를 시스템에 공급하는 방법과 예상되는 결과 유형에 대한 대안을 생각해보기에도 좋습니다.

솔루션 생성을 마쳤으면 다음 단계는 평가 지표를 가져오는 것입니다. Personalize는 훈련 데이터의 하위 집합을 기반으로 이러한 지표를 계산합니다. 아래 이미지는 Personalize를 사용하여 데이터를 분할하는 방법을 보여줍니다. 각각 10건의 상호 작용 기록이 있는 10명의 사용자가 있는 경우(원 하나는 상호 작용 하나를 나타냄) 상호 작용은 타임스탬프를 기준으로 가장 오래된 것부터 가장 최근의 것까지 순서가 매겨집니다. Personalize는 90%의 사용자(파란색 원)가 제공하는 모든 상호 작용 데이터를 사용하여 솔루션 버전을 훈련하고 나머지 10%는 평가에 사용합니다. 나머지 10%의 각 사용자에 대해 상호 작용 데이터(녹색 원)의 90%가 훈련된 모델에 대한 호출의 입력으로 사용됩니다. 나머지 10%의 데이터(주황색 원)는 모델에 의해 생성된 출력과 비교하여 평가 지표를 계산하는 데 사용됩니다.

![personalize metrics](static/imgs/personalize_metrics.png)

지표를 이해하려면 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)를 읽는 것이 좋지만, 편의를 위해 아래에 설명서의 일부를 복사해 두었습니다.

Personalize에서의 평가와 관련하여 다음 용어를 이해해야 합니다.

* 연관 추천은 특정 사용자에 대한 테스트 데이터의 값과 일치하는 추천을 말합니다.
* 순위는 추천 목록에서 추천 항목의 위치를 나타냅니다. 위치 1(목록 맨 위)은 사용자와 가장 연관성이 높은 것으로 간주됩니다.
* 쿼리는 GetRecommendations 호출에 해당하는 내부 정보를 말합니다.

Personalize에서 생성되는 지표는 다음과 같습니다.
- **coverage**: 훈련 데이터의 전체 고유 항목 수 중 모든 쿼리의 고유 추천 항목 비율(항목 및 상호 작용 데이터 세트 모두 포함)입니다.
- **mean_reciprocal_rank_at_25**: 모든 쿼리에 대한 상위 25개 추천 항목 중 첫 번째 관련 추천의 [평균 상호 순위](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)입니다. 이 지표는 순위가 가장 높은 단일 추천 항목을 파악하려는 경우 적합합니다.
- **normalized_discounted_cumulative_gain_at_K**: 절감 이득은 추천 목록에서 순위가 낮은 추천이 순위가 높은 추천보다 연관성이 적다고 가정합니다. 따라서 각 추천은 해당 위치에 따라 인수에 의해 절감됩니다(낮은 가중치가 부여됨). K에서 [누적 절감 이득](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)(DCG)을 산출하기 위해 상위 K개 추천의 각 연관 절감 추천을 함께 집계합니다. 정규화된 절감 누적 이득(NDCG)은 DCG를 이상적인 DCG로 나누어 NDCG가 0~1이 되도록 합니다. (이상적인 DCG란 상위 K 추천이 연관성에 따라 정렬하는 경우입니다.) Amazon Personalize는 1/log(1 + 위치)라는 가중치 인수를 사용합니다. 여기서 목록의 최상위는 위치 1입니다. 목록의 최상위에 있는 항목이 더 많은 주목을 받게 되므로, 이 지표는 목록 맨 위에 있는 연관 항목에 가중치를 부여합니다.
- **precision_at_K**: 상위 K개의 추천 중 연관 추천 수를 K로 나눈 값입니다. 이 지표는 연관 항목의 정확한 추천에 대해 가중치를 부여합니다.

이 노트북에서 생성된 각 솔루션의 평가 지표를 살펴보겠습니다. LastFM 데이터 세트의 품질로 인해 결과가 이 노트북의 텍스트에 설명된 결과와 다를 수 있습니다.

### AWS 사용자 개인화 지표

먼저 AWS 사용자 개인화 솔루션 버전에 대한 평가 지표를 검색합니다.

In [120]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "metrics": {
    "coverage": 0.4046,
    "mean_reciprocal_rank_at_25": 0.2035,
    "normalized_discounted_cumulative_gain_at_10": 0.2909,
    "normalized_discounted_cumulative_gain_at_25": 0.3174,
    "normalized_discounted_cumulative_gain_at_5": 0.2418,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.022,
    "precision_at_5": 0.0605
  },
  "ResponseMetadata": {
    "RequestId": "156a6e70-152b-4940-8b17-048252419fd0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:02:24 GMT",
      "x-amzn-requestid": "156a6e70-152b-4940-8b17-048252419fd0",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# 솔루션에서 캠페인 생성

## 캠페인 생성 <a class="anchor" id="create"></a>

캠페인은 호스팅된 솔루션 버전으로, 추천을 쿼리할 수 있는 엔드포인트입니다. 요금은 처리 용량(초당 사용자의 개인화 요청)을 추정하여 정해집니다. 캠페인을 배포할 때 최소 초당 처리량(TPS) 값을 설정합니다. 이 서비스는 AWS 내의 다른 서비스와 마찬가지로 수요에 따라 자동으로 확장되지만, 대기 시간이 중요한 경우에는 더 많은 수요에 대비해 미리 프로비저닝하는 것이 좋습니다. 이 POC 및 데모에서는 모든 최소 처리량 임계값을 1로 설정합니다. 자세한 내용은 [요금 페이지](https://aws.amazon.com/personalize/pricing/)를 참조하세요.

이제 캠페인을 구축해 보겠습니다.

### AWS 사용자 개인화

AWS 사용자 개인화 솔루션 버전에 대한 캠페인을 배포합니다. 캠페인을 배포하는 데 10분 정도 걸릴 수 있습니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 여러 캠페인을 만드는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 캠페인에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [122]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/airlines-metadata-campaign-14078",
  "ResponseMetadata": {
    "RequestId": "4c882630-86aa-4c5d-accd-75225f9804a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:25:37 GMT",
      "x-amzn-requestid": "4c882630-86aa-4c5d-accd-75225f9804a4",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [123]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### AWS 사용자 개인화

AWS 사용자 개인화는 Amazon Personalize에서 제공하는 고급 알고리즘 중 하나입니다. 이 알고리즘은 특정 사용자의 과거 동작을 기준으로 항목을 개인화할 수 있도록 지원하며, 다시 훈련하지 않고도 사용자에 대한 추천을 변경하기 위해 실시간 이벤트를 가져올 수 있습니다.

AWS 사용자 개인화 알고리즘은 사용자의 샘플을 사용하므로, 샘플로 사용할 데이터를 로드하고 임의의 사용자 3명을 선택합니다.

In [262]:
users_df = pd.read_csv(data_dir + '/a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
3918,JHartley,United Kingdom
27477,DDriscoll,United Kingdom
19563,BrianElliott,United Kingdom
22989,AHornbuckle,Australia
35724,CMoon,United Kingdom


이제 위에서 선택한 임의의 사용자 3명을 위한 추천을 렌더링하겠습니다. 그런 다음 개인별 순위로 넘어가기 전에 실시간 상호 작용을 살펴보도록 하겠습니다.

다시 말하지만, 결과를 멋진 데이터 프레임으로 렌더링하는 헬퍼 함수를 만듭니다.

#### API 호출 결과

In [165]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [263]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID     NATIONALITY
37013    RDow  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,thomson-airways,british-airways,thomson-airways,united-airlines
1,united-airlines,thomas-cook-airlines,virgin-atlantic-airways,virgin-atlantic-airways,british-airways
2,thomson-airways,united-airlines,turkish-airlines,air-new-zealand,american-airlines
3,virgin-atlantic-airways,easyjet,united-airlines,british-airways,china-southern-airlines
4,air-new-zealand,monarch-airlines,china-southern-airlines,united-airlines,delta-air-lines
5,turkish-airlines,virgin-atlantic-airways,qatar-airways,thomas-cook-airlines,lufthansa
6,china-southern-airlines,british-airways,emirates,turkish-airlines,alaska-airlines
7,thomas-cook-airlines,jet2-com,air-france,monarch-airlines,virgin-atlantic-airways
8,lufthansa,lufthansa,american-airlines,china-southern-airlines,us-airways
9,american-airlines,american-airlines,lufthansa,eva-air,thomson-airways


In [264]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
26198   CJeff   Singapore


,None,Economy,Business Class,Premium Economy,First Class
0,philippine-airlines,philippine-airlines,cathay-pacific-airways,cathay-pacific-airways,singapore-airlines
1,cathay-pacific-airways,singapore-airlines,philippine-airlines,air-france,thai-airways
2,singapore-airlines,tigerair,malaysia-airlines,eva-air,philippine-airlines
3,ana-all-nippon-airways,jetstar-asia,srilankan-airlines,air-new-zealand,delta-air-lines
4,thai-airways,cathay-pacific-airways,thai-airways,philippine-airlines,emirates
5,air-india,airasia,singapore-airlines,klm-royal-dutch-airlines,ana-all-nippon-airways
6,china-eastern-airlines,cebu-pacific,air-india,airasia-x,china-southern-airlines
7,dragonair,scoot,emirates,qantas-airways,alaska-airlines
8,srilankan-airlines,ana-all-nippon-airways,ana-all-nippon-airways,china-southern-airlines,american-airlines
9,air-france,dragonair,asiana-airlines,united-airlines,china-eastern-airlines


여기에서, 각 사용자마다 추천이 서로 다르다는 것을 분명히 알 수 있습니다. 이러한 결과에 캐시가 필요한 경우 먼저 모든 사용자에 대해 API 호출을 실행하고 결과를 저장하거나, 이 노트북의 뒷부분에서 다룰 배치 내보내기를 사용할 수 있습니다.

다음 주제는 실시간 이벤트입니다. Personalize에는 사용자에게 표시되는 추천을 업데이트하기 위해 프로그램에서 이벤트를 수신하는 기능이 있습니다. 이 기능은 온디맨드 비디오와 같은 미디어 워크로드에서 특히 유용합니다. 온디맨드 비디오의 경우 자녀와 함께 시청하는지 아니면 혼자 시청하는지에 따라 고객의 의도가 달라질 수 있습니다.

또한 이 시스템을 통해 기록된 이벤트는 삭제 호출이 실행될 때까지 저장되며, 다음 모델을 훈련할 때 제공하는 다른 상호 작용 데이터와 함께 기록 데이터로 사용됩니다.

#### 실시간 이벤트

캠페인에 연결되는 이벤트 트래커를 만드는 것부터 시작합니다.

In [150]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:144386903708:event-tracker/d2e7ccdc
820029aa-b00c-4eff-9e6f-60830bb68508


특정 항목과 상호 작용하는 사용자를 시뮬레이션하는 코드를 만듭니다. 이 코드를 실행한 후에는 위의 결과와 다른 추천을 받게 됩니다.

먼저 실시간 이벤트 시뮬레이션을 위한 몇 가지 메서드를 생성합니다.

In [200]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


이 시점에는 아직 실시간 이벤트를 생성하지 않았으며 코드만 설정했습니다. 실시간 이벤트 전후의 추천을 비교하기 위해, 한 명의 사용자를 선택하고 해당 사용자에 대한 원래 추천을 생성해 보겠습니다.

## 이벤트 트래커 사용 전의 추천

In [265]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID NATIONALITY
6313  ACrociani       Italy


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,alitalia,iberia,british-airways,american-airlines
1,alitalia,brussels-airlines,british-airways,virgin-atlantic-airways,british-airways
2,iberia,ryanair,qatar-airways,united-airlines,delta-air-lines
3,brussels-airlines,easyjet,alitalia,brussels-airlines,united-airlines
4,qatar-airways,iberia,brussels-airlines,turkish-airlines,lufthansa
5,lufthansa,aer-lingus,emirates,alitalia,emirates
6,american-airlines,aegean-airlines,lufthansa,air-france,qatar-airways
7,united-airlines,lufthansa,turkish-airlines,lufthansa,swiss-international-air-lines
8,icelandair,qatar-airways,swiss-international-air-lines,icelandair,us-airways
9,delta-air-lines,tap-portugal,oman-air,delta-air-lines,iberia


In [266]:
user_id

'ACrociani'

실시간 이벤트를 적용하기 전의 이 사용자에 대한 추천 목록이 있습니다. 이제 사용자의 상호 작용을 시뮬레이션할 임의의 아티스트 3명을 선택하고 그에 따라 추천이 어떻게 바뀌는지 살펴보겠습니다.

In [267]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['thai-airways', 'austrian-airlines', 'united-airlines']

In [268]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

ACrociani


,thai-airways|Economy,austrian-airlines|Economy,united-airlines|Economy
0,alitalia,ryanair,brussels-airlines
1,brussels-airlines,brussels-airlines,lufthansa
2,ryanair,easyjet,turkish-airlines
3,easyjet,tap-portugal,austrian-airlines
4,iberia,aegean-airlines,tap-portugal
5,aer-lingus,turkish-airlines,alitalia
6,aegean-airlines,lufthansa,germanwings
7,lufthansa,alitalia,aegean-airlines
8,qatar-airways,iberia,swiss-international-air-lines
9,tap-portugal,air-india,air-berlin


위의 셀에서 인덱스 다음의 첫 번째 열은 AWS 사용자 개인화 모델에서 사용자의 기본 추천이며, 그 다음의 각 열에는 실시간 이벤트를 통해 상호 작용한 항공사의 헤더와 이 이벤트가 발생한 후의 추천이 있습니다.

이 동작은 크게 바뀌지 않을 수 있습니다. 이는 비교적 제한적인 이 데이터 세트의 특성 때문입니다. 이를 더 잘 확실하게 알아보고 싶다면, 무작위로 항공사에 평점을 매겨 봅니다. 그러면 그 영향이 더 뚜렷하게 나타납니다.